In [3]:
# !pip install imblearn

# Import

In [9]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split

import pandas as pd

# ML

In [10]:
# 1차 EDA 불러오기
df_diabetes = pd.read_csv('./data/df_diabetes.csv')

### SMOTE 기법

In [14]:
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# 1. 입력 변수와 타겟 정의
features = [
    'HE_DM_HbA1c', 'HE_glu', 'HE_Uglu', 'HE_HbA1c', 'HE_hsCRP',
    'HE_chol', 'HE_HDL_st2', 'HE_BUN', 'HE_TG', 'HE_LDL_drct',
    'HE_wc', 'HE_BMI',
    'SBP_mean','DBP_mean',
]

X = df_diabetes[features]
y = df_diabetes['diabetes_label'] 

# 2. train/test 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. 결측치 평균값으로 채우기
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# 4. SMOTE 적용 (훈련셋만)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_imputed, y_train)


### SMOTE 성능평가

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. 모델 학습
model = RandomForestClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

# 2. 테스트셋 예측
y_pred = model.predict(X_test_imputed)

# 3. 성능 평가
print("정확도 (Accuracy):", accuracy_score(y_test, y_pred))
print("\n 분류 리포트:\n", classification_report(y_test, y_pred, target_names=['비당뇨(0)', '당뇨(1)']))
print("\n 혼동 행렬:\n", confusion_matrix(y_test, y_pred))

# [[1225   28]   → 28명은 당뇨 아닌데 당뇨라고 예측 (위양성)
#  [  4  129]]   → 4명은 실제 당뇨인데 놓쳤음 (위음성)


✅ 정확도 (Accuracy): 0.976911976911977

📊 분류 리포트:
               precision    recall  f1-score   support

      비당뇨(0)       1.00      0.98      0.99      1253
       당뇨(1)       0.82      0.97      0.89       133

    accuracy                           0.98      1386
   macro avg       0.91      0.97      0.94      1386
weighted avg       0.98      0.98      0.98      1386


🧩 혼동 행렬:
 [[1225   28]
 [   4  129]]


### SMOTETomek 기법